<a href="https://colab.research.google.com/github/demichie/CorsoIntroduzionePython/blob/main/lezione6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lezione 6: Analisi di Dati Tabulari con Pandas**

Nelle lezioni precedenti abbiamo costruito una solida base: abbiamo imparato la sintassi di Python, come organizzare i dati in liste e dizionari, e come eseguire calcoli numerici efficienti con **NumPy**. Infine, con **Matplotlib**, abbiamo visto come trasformare i nostri array di numeri in visualizzazioni informative.

Oggi facciamo il passo successivo e cruciale per ogni ricercatrice e ricercatore che deve analizzare dei dati: impariamo a lavorare con dati **tabulari**. I dati nel mondo reale raramente sono semplici griglie di numeri; più spesso, si presentano in formati strutturati come fogli di calcolo o tabelle, con righe che rappresentano osservazioni (es. un campione di roccia) e colonne che rappresentano variabili (es. SiO₂, MgO, Località), complete di etichette e tipi di dati misti (numeri, testo, date).

Per gestire questo tipo di dati, useremo **Pandas**, la libreria più importante in Python per l'analisi e la manipolazione di dati tabulari.

### **Obiettivi dettagliati della Lezione 6:**
*   Comprendere il ruolo di **Pandas** e la sua struttura dati centrale: il **DataFrame**.
*   Imparare a **leggere dati** da file esterni, in particolare file `.csv`, usando `pd.read_csv()`.
*   Sviluppare un flusso di lavoro standard per **ispezionare** un DataFrame appena caricato (`.head()`, `.info()`, `.describe()`).
*   **Pulire** i dati, a partire dalla standardizzazione dei nomi delle colonne.
*   Imparare le tecniche fondamentali per **selezionare e filtrare** i dati: per colonna, per posizione/etichetta di riga (`.iloc`, `.loc`) e tramite condizioni (`masking booleano`).
*   **Creare nuove colonne** basate su calcoli tra quelle esistenti.

## **Perché Pandas? Oltre gli Array NumPy**

Pandas è una libreria costruita sopra NumPy e ne sfrutta la potenza per i calcoli veloci. Tuttavia, aggiunge due strutture dati fondamentali che risolvono i limiti degli array "nudi":

1.  **La `Series`**: Possiamo pensarla come una singola colonna di una tabella. È essenzialmente un array NumPy 1D a cui è associato un **indice** (una serie di etichette). Questo indice ci permette di accedere ai dati in modo più flessibile e significativo rispetto alla sola posizione numerica.

2.  **Il `DataFrame`**: Questa è la struttura dati centrale di Pandas e sarà il nostro principale strumento di lavoro. Un DataFrame è una **tabella a 2 dimensioni**, simile a un foglio di calcolo o a una tabella di un database.

    *   È composto da più `Series` (le colonne), ognuna con il proprio nome.
    *   Le colonne possono avere tipi di dati diversi (es. una colonna di testo per i nomi, una di numeri per le elevazioni).
    *   Condivide un unico **indice** per le righe, che permette di allineare i dati in modo coerente.

In pratica, il DataFrame aggiunge un contesto ricco (etichette di riga e colonna, tipi di dati eterogenei) alla velocità di calcolo di NumPy.

In [ ]:
# Importiamo le librerie che useremo in questa lezione.
# La convenzione standard è importare pandas con l'alias 'pd'.
# Useremo anche NumPy, dato che Pandas è costruito su di esso.
# Per questa lezione, useremo 'kagglehub' per scaricare i dati e 'os'
# per interagire con il file system.

import pandas as pd
import numpy as np
import kagglehub
import os

print("Librerie importate con successo!")

## **Parte 1: Caricare i Dati: File Locali e Risorse Online**

Il primo passo di ogni analisi dati è importare i dati nel nostro ambiente di lavoro. Tipicamente, i dati si trovano in file (come `.csv` o fogli Excel) salvati sul nostro computer. In quel caso, passeremmo a Pandas il percorso locale del file.

Tuttavia, per rendere questa lezione facilmente riproducibile per tutti e per lavorare con un dataset interessante e reale, scaricheremo i nostri dati direttamente da **Kaggle**.

**Cos'è Kaggle?**
[Kaggle](https://www.kaggle.com/) è una piattaforma online popolarissima per la data science. È un luogo dove si possono trovare migliaia di dataset pubblici, partecipare a competizioni di machine learning e condividere analisi. È una risorsa inestimabile per fare pratica.

Per interagire con Kaggle direttamente dal nostro notebook, useremo la libreria `kagglehub`.

**Il Nostro Dataset: Eruzioni Vulcaniche dal GVP**
Il dataset che useremo è il catalogo delle eruzioni vulcaniche del **Global Volcanism Program (GVP)**, curato dalla Smithsonian Institution. Contiene un'enorme quantità di informazioni sui vulcani della Terra e la loro attività. Potete trovare la pagina web del dataset [qui](https://www.kaggle.com/datasets/smithsonian/volcanic-eruptions).

Useremo `kagglehub` per scaricare questo dataset. La funzione si occuperà di scaricare i file e scompattarli in una cartella locale, pronta per essere esplorata.

### **Uno Strumento Essenziale: Interagire con il Sistema Operativo con la Libreria `os`**

Una volta che `kagglehub` avrà scaricato i dati, questi si troveranno in una cartella sul nostro sistema. Per poterli leggere, dobbiamo prima capire **dove** sono e **quali file** contiene la cartella.

Per fare questo, usiamo la libreria standard di Python `os`. Il suo nome sta per "Operating System" (Sistema Operativo), e ci fornisce gli strumenti per interagire con il file system in modo **indipendente dalla piattaforma** (funziona allo stesso modo su Windows, macOS e Linux).

Ci concentreremo su due funzioni principali:

1.  **`os.listdir(percorso)`**: Restituisce una **lista** con i nomi di tutti i file e le sottocartelle presenti in un dato `percorso`. È il nostro strumento per "guardare dentro" una cartella.

2.  **`os.path.join(percorso1, percorso2, ...)`**: È il modo **corretto e sicuro** per costruire un percorso di file. Unisce le stringhe usando il separatore giusto per il sistema operativo in uso (`/` o `\`), rendendo il nostro codice portabile.

In [ ]:
# --- Esempio 1: Esplorare una cartella di sistema ---

# Definiamo il percorso di una cartella. In Colab, '/content/' è la cartella di lavoro principale.
# Questa cartella di solito contiene una sottocartella 'sample_data'.
path_to_explore = '/content/'

print(f"Esplorazione della cartella: '{path_to_explore}'")

try:
    # Usiamo os.listdir per vedere cosa c'è dentro
    content_list = os.listdir(path_to_explore)
    print("Contenuto trovato:")
    for item in content_list:
        print(f"- {item}")
except FileNotFoundError:
    print(f"La cartella '{path_to_explore}' non esiste.")

print("\n" + "-"*50 + "\n")

In [ ]:
# --- Esempio 2: Costruire un percorso in modo sicuro ---

# Definiamo le parti di un percorso. Immaginiamo di voler accedere a un file
# che si trova dentro la cartella 'sample_data' vista prima.
folder_name = 'sample_data'
file_name = 'granulometrie.csv' # Un file di esempio in Colab

# Usiamo os.path.join per creare il percorso completo
full_path = os.path.join(path_to_explore, folder_name, file_name)

print(f"Percorso costruito con os.path.join: '{full_path}'")

# Nota: os.path.join si occupa della compatibilità per noi. Se eseguissimo questo
# su Windows, l'output userebbe il backslash '\\' come separatore.

### **Applicazione Pratica: Troviamo e Prepariamo i Nostri Dati**

Ora applicheremo gli strumenti della libreria `os` che abbiamo appena visto per creare un flusso di lavoro di caricamento dati robusto.

La nostra strategia sarà la seguente:

1.  **Download:** Useremo `kagglehub.dataset_download()` per scaricare e scompattare il dataset in una cartella locale. Salveremo il percorso di questa cartella in una variabile.
2.  **Esplorazione:** Useremo `os.listdir()` per ispezionare il contenuto di questa cartella e identificare i file `.csv` disponibili.
3.  **Preparazione:** Successivamente, nelle celle che seguono, useremo `os.path.join()` per costruire il percorso completo del file che ci interessa e `pd.read_csv()` per caricarlo in un DataFrame.

Questo approccio in più passaggi è un esempio di "programmazione difensiva": non diamo per scontato dove siano i file o come si chiamino, ma lo scopriamo in modo programmatico.

In [ ]:
# --- 1. Download del Dataset ---
print("Download del dataset 'smithsonian/volcanic-eruptions' dal KaggleHub...")
dataset_path = kagglehub.dataset_download("smithsonian/volcanic-eruptions")
print("Download completato.")
print(f"I file del dataset si trovano nella cartella: {dataset_path}\n")


# --- 2. Esplorazione del Contenuto della Cartella ---
# Usiamo os.listdir per vedere quali file sono stati scaricati e identificare
# i file .csv che ci interessano.
print("File trovati nella cartella del dataset:")
try:
    available_files = os.listdir(dataset_path)
    # Creiamo una lista contenente solo i file che terminano con .csv
    csv_files = [file for file in available_files if file.endswith('.csv')]

    for file_name in available_files:
        print(f"- {file_name}")

    if not csv_files:
        print("\nATTENZIONE: Nessun file .csv trovato nella cartella del dataset.")

except FileNotFoundError:
    print(f"ERRORE: La cartella del dataset '{dataset_path}' non è stata trovata.")
    csv_files = [] # Resetta a lista vuota in caso di errore

### **Caricare Dati con `pd.read_csv()`: Potenza e Flessibilità**

Ora che abbiamo identificato i file `.csv` nel nostro dataset, è il momento di caricarli in un DataFrame. La funzione `pd.read_csv()` è lo strumento principale per importare dati tabulari in Pandas. Sebbene il nome suggerisca che funzioni solo con file "CSV" (Comma-Separated Values), in realtà è un parser di file di testo estremamente potente e configurabile.

**Quali tipi di file possiamo leggere?**
`pd.read_csv()` può gestire quasi ogni file di testo strutturato in colonne, a patto di specificare le opzioni corrette. Questo include file `.tsv` (separati da tab), file con separatori personalizzati (`;`, `|`), file senza intestazione, e molto altro.

**Le Opzioni Più Comuni di `read_csv()`**
Per gestire questa varietà di formati, `pd.read_csv()` accetta decine di argomenti opzionali. Vediamo i più importanti in una tabella riassuntiva.

| Argomento          | Descrizione                                                                                                   | Esempio d'uso                                          |
|--------------------|---------------------------------------------------------------------------------------------------------------|--------------------------------------------------------|
| `filepath_or_buffer` | **(Obbligatorio)** Il percorso del file locale, un URL, o un oggetto file.                                       | `'data/mio_file.csv'` o `'http://.../data.csv'`      |
| `sep` (o `delimiter`)| Il carattere usato per separare i valori. Di default è la virgola (`,`).                                        | `sep='\t'` (per file .tsv) o `sep=';'`                   |
| `header`           | Indica quale riga del file usare come intestazione. Di default è `0` (la prima riga).                           | `header=None` (se il file non ha intestazione)         |
| `names`            | Una lista di nomi di colonna da usare, tipicamente con `header=None`.                                         | `names=['col1', 'col2']`                               |
| `index_col`        | Indica quale colonna usare come indice del DataFrame.                                                          | `index_col=0` (la prima colonna) o `index_col='ID'`      |
| `usecols`          | Una lista di nomi o indici di colonna per leggere solo un sottoinsieme delle colonne.                           | `usecols=['col1', 'col3']`                             |
| `skiprows`         | Un numero intero per saltare le prime `n` righe del file (utile per saltare commenti).                         | `skiprows=5`                                           |
| `nrows`            | Un numero intero per leggere solo le prime `n` righe di dati (ottimo per file molto grandi).                   | `nrows=1000`                                           |
| `na_values`        | Una stringa o una lista di stringhe da riconoscere come valori mancanti (NaN).                                 | `na_values=['N/A', '-999']`                            |
| `decimal`          | Il carattere da usare come separatore decimale (es. in Europa si usa la virgola).                               | `decimal=','`                                          |

**Esempio pratico:**
Se avessimo un file `dati.txt` separato da punto e virgola, senza header, e con la virgola come separatore decimale, lo caricheremmo così:

```python
# df_esempio = pd.read_csv(
#     'dati.txt',
#     sep=';',
#     header=None,
#     names=['temperatura', 'pressione'],
#     decimal=','
# )
```

Per la nostra lezione, i file `.csv` del GVP sono ben formattati, quindi l'uso base di `pd.read_csv()` sarà sufficiente.

In [ ]:
# --- Caricamento del primo file CSV trovato ---

# Inizializziamo il nostro DataFrame come vuoto.
df = pd.DataFrame()
file_loaded = "" # Una variabile per tenere traccia del nome del file caricato

# La variabile 'csv_files' (la lista dei file CSV) è stata creata nella cella precedente.
# Controlliamo se la lista contiene almeno un file.
if csv_files:
    # Selezioniamo il primo file CSV dalla lista. NESSUNA ASSUNZIONE SUL NOME.
    file_to_load = csv_files[0]

    # Costruiamo il percorso completo del file
    file_path = os.path.join(dataset_path, file_to_load)

    print(f"Trovati file CSV. Caricamento del primo della lista: '{file_path}'")

    try:
        # Usiamo pd.read_csv per caricare il file
        df = pd.read_csv(file_path)
        file_loaded = file_to_load # Salviamo il nome del file caricato
        print("Caricamento completato con successo!")

    except Exception as e:
        print(f"\nERRORE durante il caricamento del file: {e}")
        # 'df' rimarrà vuoto come inizializzato

else:
    print("\nCaricamento saltato: nessun file .csv trovato nella cartella del dataset.")

### **Nota: Lavorare con Altri Formati - `pd.read_excel()`**

Sebbene `read_csv()` sia la funzione più comune, nel lavoro di tutti i giorni vi capiterà spessissimo di avere dati in **fogli di calcolo Excel (`.xls` o `.xlsx`)**.

Pandas gestisce questo formato in modo altrettanto semplice con la funzione `pd.read_excel()`.

**Sintassi di base:**
```python
# df_excel = pd.read_excel('percorso/del/mio_file.xlsx')
```

La funzione `read_excel` è molto potente e accetta argomenti specifici per i file Excel, come:
*   `sheet_name`: Per specificare quale foglio di lavoro leggere (per nome o per indice). Di default legge il primo. `sheet_name='Dati_Greggi'` o `sheet_name=1`.
*   `skiprows`, `header`, `usecols`, ecc.: Molti degli argomenti che abbiamo visto per `read_csv` funzionano anche qui.

**Requisiti di Installazione:**
A differenza di `read_csv`, per leggere file Excel, Pandas ha bisogno di una libreria aggiuntiva (un "motore"). Le più comuni sono **`openpyxl`** (per file `.xlsx`) e **`xlrd`** (per i vecchi file `.xls`).

Se provate a eseguire `pd.read_excel()` senza averle installate, Pandas vi darà un errore molto chiaro, suggerendovi il comando da eseguire per installarle, ad esempio:
`pip install openpyxl`

Per questa lezione continueremo a usare il nostro file `.csv`, ma è fondamentale sapere che Pandas è perfettamente equipaggiato per interagire direttamente con i vostri file Excel.

## **Parte 2: Ispezione e Pulizia Iniziale del DataFrame**

Abbiamo caricato i nostri dati in un DataFrame Pandas chiamato `df`. Ma cosa contiene esattamente? È stato caricato correttamente? Quante righe ha? Quali sono le colonne?

Prima di iniziare qualsiasi analisi, il primo passo fondamentale è sempre **ispezionare** il DataFrame per capirne la struttura e il contenuto. Pandas ci offre una serie di comandi semplici e potenti per questa "ricognizione" iniziale.

### **Controllo Preliminare: l'attributo `.empty`**

Prima ancora di guardare i dati, è una buona pratica verificare se il caricamento ha avuto successo e se il DataFrame contiene effettivamente qualcosa. Potrebbe essere vuoto se il file di origine era vuoto o se si è verificato un errore che abbiamo gestito.

Per fare questo controllo, usiamo l'attributo `.empty`.
*   `df.empty` restituisce `True` se il DataFrame non ha righe.
*   `df.empty` restituisce `False` se il DataFrame contiene almeno una riga.

Useremo questo attributo per assicurarci di non provare a eseguire operazioni su un DataFrame vuoto.

### **Prima Occhiata ai Dati: `.head()`**

Una volta sicuri che il DataFrame non è vuoto, il primo comando da lanciare è quasi sempre `.head()`. Questo metodo ci mostra le **prime 5 righe** del DataFrame. È il modo più rapido per avere un'idea visiva di:
*   I nomi delle colonne.
*   Il tipo di dati in ogni colonna (numeri, testo, ecc.).
*   La struttura generale della tabella.

È possibile specificare un numero diverso di righe da visualizzare passandolo come argomento, ad esempio `df.head(10)` per vedere le prime 10 righe.

In [ ]:
# Controlliamo se il DataFrame non è vuoto prima di provare a ispezionarlo
if not df.empty:
    print(f"Visualizzazione delle prime 5 righe del file '{file_loaded}':")
    # La funzione display() di Jupyter formatta l'output dei DataFrame in modo molto leggibile
    display(df.head())
else:
    print("Il DataFrame è vuoto, non c'è nulla da visualizzare.")

### **Ottenere un Riepilogo Tecnico e Statistico**

Oltre a `.head()` per un'ispezione visiva, Pandas offre due metodi potentissimi per ottenere un riepilogo quantitativo del nostro DataFrame.

#### **Riepilogo Tecnico: `.info()`**
Il metodo `.info()` fornisce una sintesi concisa della struttura del DataFrame. È fondamentale per capire:
*   Il numero totale di righe (Entries).
*   L'elenco di tutte le colonne.
*   Il numero di valori **non nulli** per ogni colonna. Questa è la prima e più importante indicazione della presenza di **dati mancanti** (`missing values`).
*   Il tipo di dato (`Dtype`) di ogni colonna (es. `int64` per interi, `float64` per numeri decimali, `object` per stringhe o dati misti).
*   L'utilizzo totale della memoria.

#### **Riepilogo Statistico: `.describe()`**
Il metodo `.describe()` calcola automaticamente le principali statistiche descrittive per tutte le **colonne numeriche**:
*   `count`: Il numero di valori non nulli.
*   `mean`: La media.
*   `std`: La deviazione standard.
*   `min`: Il valore minimo.
*   `25%`, `50%`, `75%`: I percentili (il 50° percentile è la mediana).
*   `max`: Il valore massimo.

È un comando incredibilmente utile per avere una prima idea della distribuzione e della scala dei nostri dati numerici.

In [ ]:
# Usiamo .info() per ottenere un riepilogo tecnico del nostro DataFrame
if not df.empty:
    print("Riepilogo tecnico del DataFrame:")
    df.info()
else:
    print("Il DataFrame è vuoto.")

# PUNTO DI DISCUSSIONE:
# Dall'output, cosa possiamo dedurre? Notiamo subito che le colonne 'Dominant Rock Type'
# e 'Tectonic Setting' hanno meno valori non-nulli rispetto al totale delle righe.
# Questo ci conferma la presenza di dati mancanti!

In [ ]:
# Usiamo .describe() per le statistiche delle colonne numeriche
if not df.empty:
    print("Riepilogo statistico delle colonne numeriche:")
    # display() formatta bene anche l'output di .describe()
    display(df.describe())
else:
    print("Il DataFrame è vuoto.")

# PUNTO DI DISCUSSIONE:
# Qualche osservazione interessante?
# - La colonna 'Number' sembra un ID e non ha molto significato statistico.
# - 'Elevation (Meters)' varia da valori negativi (vulcani sottomarini) a molto alti.
# - La media della latitudine è positiva, suggerendo più vulcani nell'emisfero nord nel dataset.

### **Pulizia dei Dati: Standardizzare i Nomi delle Colonne**

Dall'ispezione con `.head()` e `.info()`, abbiamo notato che i nomi delle colonne nel nostro DataFrame non sono ideali per la programmazione: contengono spazi (es. `'Activity Evidence'`), parentesi (es. `'Elevation (Meters)'`) e un mix di lettere maiuscole e minuscole.

Questi nomi "scomodi" possono causare problemi:
*   Non possiamo usare la notazione ad attributo (es. `df.Activity Evidence` darebbe un errore di sintassi).
*   Sono più lunghi da scrivere e più facili da sbagliare (`df['Elevation (Meters)']`).

Una delle primissime fasi della pulizia dei dati (`data cleaning`) è quasi sempre la **standardizzazione dei nomi delle colonne**. Una convenzione molto comune, nota come "snake_case", prevede di rendere tutti i caratteri minuscoli e di sostituire gli spazi con un trattino basso (`_`).

Pandas ci offre diversi modi per farlo. Vediamo un approccio in due passaggi.

#### **Passaggio 1: Rinominare Colonne Specifiche con `.rename()`**

Per modifiche mirate, il metodo `.rename()` è lo strumento perfetto. Accetta un argomento `columns` a cui passiamo un **dizionario Python**.
*   Le **chiavi** del dizionario sono i nomi delle colonne attuali che vogliamo cambiare.
*   I **valori** del dizionario sono i nuovi nomi che vogliamo assegnare.

**L'argomento `inplace=True`**
Per default, i metodi di manipolazione di Pandas restituiscono una **nuova copia** del DataFrame con le modifiche applicate, lasciando l'originale intatto. Se vogliamo modificare il nostro DataFrame "sul posto" (cioè direttamente, senza creare una copia), possiamo usare l'argomento `inplace=True`. È una scelta di convenienza, ma bisogna usarla con attenzione.

```python
# Sintassi di esempio
# df.rename(columns={'Vecchio Nome': 'nuovo_nome'}, inplace=True)
```



In [ ]:
# Passaggio 1: Usare il metodo .rename() per correggere le colonne più complesse.
# .rename() accetta un dizionario dove le chiavi sono i nomi attuali e i valori sono i nuovi nomi.

if not df.empty:
    print("Nomi delle colonne PRIMA della rinomina:")
    print(df.columns.tolist()) # .columns.tolist() ci dà una lista pulita dei nomi

    df.rename(columns={
        'Elevation (Meters)': 'elevation_meters',
        'Activity Evidence': 'activity_evidence',
        'Last Known Eruption': 'last_known_eruption',
        'Dominant Rock Type': 'dominant_rock_type',
        'Tectonic Setting': 'tectonic_setting'
    }, inplace=True) # inplace=True modifica il DataFrame direttamente, senza bisogno di fare df = ...

    print("\nNomi delle colonne DOPO la prima rinomina:")
    print(df.columns.tolist())
else:
    print("Il DataFrame è vuoto, nessuna colonna da rinominare.")

#### **Passaggio 2: Applicare una Trasformazione a Tutti i Nomi**

Per operazioni sistematiche su tutti i nomi (come renderli tutti minuscoli o sostituire tutti gli spazi), è inefficiente usare `.rename()`. Un approccio molto più potente è:
1.  Ottenere la lista di tutti i nomi delle colonne con `df.columns`.
2.  Usare una **list comprehension** per creare una nuova lista con i nomi trasformati (es. usando i metodi delle stringhe `.lower()` e `.replace()`).
3.  Assegnare questa nuova lista direttamente a `df.columns`.

Questo pattern è estremamente comune e potente per la pulizia dei dati.


In [ ]:
# Passaggio 2: Rendiamo tutto minuscolo e sostituiamo gli spazi rimanenti.
# Un modo potente e "Pythonico" per farlo è usare una list comprehension per
# creare una nuova lista di nomi e assegnarla a df.columns.

if not df.empty:
    # Per ogni nome di colonna in df.columns, lo rendiamo minuscolo e sostituiamo ' ' con '_'
    nuovi_nomi = [col.lower().replace(' ', '_') for col in df.columns]

    # Assegniamo la nuova lista di nomi all'attributo .columns del DataFrame
    df.columns = nuovi_nomi

    print("Nomi delle colonne finali e standardizzati:")
    print(df.columns.tolist())

    # Verifichiamo il risultato finale con .head()
    print("\nDataFrame con i nuovi nomi di colonna:")
    display(df.head())
else:
    print("Il DataFrame è vuoto.")

### **Digressione: I Metodi delle Stringhe, i Nostri Alleati per la Pulizia dei Dati**

Nella cella precedente, abbiamo usato `col.lower().replace(' ', '_')` per trasformare i nomi delle colonne. Questa "catena" di comandi è un esempio perfetto di come si usano i **metodi delle stringhe** in Python.

Un metodo è una funzione che "appartiene" a un oggetto. Le stringhe in Python hanno decine di metodi incredibilmente utili per manipolare il testo. Questi sono fondamentali nel *data cleaning*, perché i dati testuali nel mondo reale sono spesso "sporchi": contengono spazi extra, maiuscole/minuscole inconsistenti, caratteri indesiderati, ecc.

Ripassiamo e introduciamo alcuni dei più importanti:

| Metodo                 | Descrizione                                                                              | Esempio d'uso                                            |
|------------------------|------------------------------------------------------------------------------------------|----------------------------------------------------------|
| `.lower()`             | Converte tutti i caratteri della stringa in minuscolo.                                     | `'Etna'.lower()` -> `'etna'`                             |
| `.upper()`             | Converte tutti i caratteri della stringa in maiuscolo.                                     | `'Etna'.upper()` -> `'ETNA'`                             |
| `.strip()`             | Rimuove gli spazi (o altri caratteri specificati) dall'**inizio e dalla fine** della stringa. | `'  Vesuvio  '.strip()` -> `'Vesuvio'`                  |
| `.replace(vecchio, nuovo)` | Sostituisce tutte le occorrenze della sottostringa `vecchio` con `nuovo`.                 | `'Last Known Eruption'.replace(' ', '_')` -> `'Last_Known_Eruption'` |
| `.split(separatore)`   | Divide la stringa in una **lista** di sottostringhe, usando il `separatore` come punto di divisione. | `'45.5,8.5'.split(',')` -> `['45.5', '8.5']`        |
| `.startswith(testo)`   | Restituisce `True` se la stringa inizia con `testo`, altrimenti `False`.                    | `'Volcano_Etna'.startswith('Volcano')` -> `True`         |
| `.endswith(testo)`     | Restituisce `True` se la stringa finisce con `testo`, altrimenti `False`.                    | `'data.csv'.endswith('.csv')` -> `True`                  |

Questi metodi possono essere **concatenati**: l'output di un metodo diventa l'input del successivo, permettendo trasformazioni complesse in una sola riga, come abbiamo fatto per i nomi delle colonne.

#### **Esercizio Pratico: Pulire una Stringa di Dati**

Immaginiamo di aver ricevuto una riga di dati da un vecchio file di testo, formattata come una singola stringa. Il nostro compito è estrarre e pulire le informazioni.

**Dato di partenza:**
Una stringa che contiene: ID del campione, tipo di roccia e percentuale di SiO₂, separati da punto e virgola e con spazi disordinati.
`riga_dati = "  SAMP-001 ; Basalt ; SiO2: 50.25%  "`

**Compiti:**
1.  **Pulizia Iniziale:** Usa `.strip()` per rimuovere gli spazi inutili all'inizio e alla fine della stringa `riga_dati`.
2.  **Suddivisione:** Usa `.split(';')` sulla stringa pulita per ottenere una lista di tre elementi. Salva il risultato in una variabile `dati_lista`.
3.  **Pulizia degli Elementi:** Ora, per ogni elemento nella `dati_lista`, applica di nuovo `.strip()` per rimuovere gli spazi attorno a ogni valore. (Suggerimento: puoi farlo con una list comprehension!).
4.  **Estrazione del Valore Numerico:** Prendi l'ultimo elemento della lista (es. `'SiO2: 50.25%'`). Usa `.replace()` due volte per rimuovere prima `'SiO2: '` e poi `'%'`. Infine, converti la stringa risultante in un numero `float`.
5.  **Stampa i risultati finali:** Stampa l'ID del campione pulito, il tipo di roccia (in minuscolo) e il valore numerico di SiO₂.

In [ ]:
# Dato di partenza
riga_dati = "  SAMP-001 ; Basalt ; SiO2: 50.25%  "

# 1. Pulizia Iniziale
# Scrivi qui il tuo codice


# 2. Suddivisione in una lista
# Scrivi qui il tuo codice


# 3. Pulizia di ogni elemento nella lista
# Scrivi qui il tuo codice


# 4. Estrazione e conversione del valore di SiO2
# Scrivi qui il tuo codice


# 5. Stampa dei risultati finali
# Scrivi qui il tuo codice (es. print(f"ID: {id_campione}, Tipo: {tipo_roccia}, SiO2: {sio2_valore}"))

In [ ]:
# Dato di partenza
riga_dati = "  SAMP-001 ; Basalt ; SiO2: 50.25%  "


In [ ]:
# --- Soluzione ---

# 1. Pulizia Iniziale
# Usiamo .strip() per rimuovere gli spazi bianchi all'inizio e alla fine.
riga_pulita = riga_dati.strip()
print(f"1. Dopo .strip() iniziale: '{riga_pulita}'")


In [ ]:
# 2. Suddivisione in una lista
# Usiamo .split(';') per dividere la stringa in base al punto e virgola.
dati_lista = riga_pulita.split(';')
print(f"2. Dopo .split(';'): {dati_lista}")


In [ ]:
# 3. Pulizia di ogni elemento nella lista
# Notiamo che ogni elemento ha ancora spazi attorno. Usiamo una list comprehension
# per applicare .strip() a ogni elemento della lista.
dati_lista_pulita = [elemento.strip() for elemento in dati_lista]
print(f"3. Dopo la pulizia di ogni elemento: {dati_lista_pulita}")


In [ ]:
# 4. Estrazione e conversione del valore di SiO2
# Prendiamo l'ultimo elemento della lista, che è la stringa del SiO2.
sio2_stringa = dati_lista_pulita[2]

# Usiamo una catena di .replace() per rimuovere il testo non numerico.
# Primo replace: rimuove 'SiO2: ' (nota lo spazio)
sio2_val_str = sio2_stringa.replace('SiO2: ', '')
# Secondo replace: rimuove il simbolo '%'
sio2_val_str = sio2_val_str.replace('%', '')

print(f"4. Stringa SiO2 dopo le sostituzioni: '{sio2_val_str}'")

# Infine, convertiamo la stringa pulita in un numero float.
sio2_valore = float(sio2_val_str)
print(f"   Valore SiO2 convertito in float: {sio2_valore}")

In [ ]:
# 5. Stampa dei risultati finali
# Estraiamo gli altri valori dalla lista pulita e formattiamo l'output.
id_campione = dati_lista_pulita[0]
tipo_roccia = dati_lista_pulita[1].lower() # Lo mettiamo in minuscolo come da richiesta

print("\n--- Risultati Finali ---")
print(f"ID Campione: {id_campione}")
print(f"Tipo Roccia: {tipo_roccia}")
print(f"Valore SiO2: {sio2_valore}")
print(f"Tipo di dato del valore SiO2: {type(sio2_valore)}")

## **Parte 3: Selezionare e Filtrare i Dati**

Ora che il nostro DataFrame `df` ha nomi di colonna puliti e standardizzati, possiamo iniziare la fase di analisi vera e propria: la **selezione dei dati**. Raramente siamo interessati a tutto il dataset in una volta; più spesso, vogliamo isolare specifiche colonne o righe che soddisfano determinati criteri.

### **Selezione di Colonne**

Ci sono due modi principali per selezionare una o più colonne da un DataFrame.

**1. Selezionare una Singola Colonna**
Per selezionare una singola colonna, si usa una sintassi simile a quella dei dizionari Python, passando il nome della colonna come stringa tra parentesi quadre. Il risultato è un oggetto **Pandas Series**.

```python
# Sintassi: df['nome_colonna']
```
Pandas offre anche una scorciatoia, la "notazione ad attributo" (`df.nome_colonna`), ma funziona solo se il nome della colonna non contiene spazi o caratteri speciali. Ora che abbiamo pulito i nostri nomi, potremmo usarla, ma la notazione con le parentesi quadre è universalmente più sicura e raccomandata.

**2. Selezionare Più Colonne**
Per selezionare più colonne, passiamo una **lista di nomi di colonna** all'interno delle parentesi quadre. Il risultato è un nuovo **DataFrame** contenente solo le colonne specificate.

**Attenzione alla sintassi:** la "lista dentro le parentesi" risulta in **doppie parentesi quadre** `[[...]]`. Questo è un errore comune per chi inizia.
```python
# Sintassi: df[['colonna_1', 'colonna_2']]
```


In [ ]:
# Assicuriamoci che il DataFrame 'df' esista e non sia vuoto
if not df.empty:

    # --- 1. Selezionare una singola colonna ('country') ---
    # Il risultato è una Series
    countries = df['country']

    print("--- Selezione della colonna 'country' (una Series) ---")
    display(countries.head())
    print(f"Tipo di oggetto restituito: {type(countries)}\n")

    # --- 2. Selezionare più colonne ('name', 'type', 'elevation_meters') ---
    # Il risultato è un nuovo DataFrame
    subset_df = df[['name', 'type', 'elevation_meters']]

    print("--- Selezione di più colonne (un nuovo DataFrame) ---")
    display(subset_df.head())
    print(f"Tipo di oggetto restituito: {type(subset_df)}")

else:
    print("Il DataFrame è vuoto. Impossibile selezionare colonne.")

### **Digressione: Ispezione Visiva di una Colonna Numerica**

Nella cella precedente abbiamo visto come estrarre una singola colonna (una `Series` Pandas). Quando la colonna contiene dati numerici, possiamo usare le competenze di **Matplotlib** che abbiamo già acquisito per farci subito un'idea della sua distribuzione.

Uno dei primi passi nell'analisi esplorativa di una nuova variabile numerica è visualizzarne la distribuzione tramite un **istogramma**. Questo ci aiuta a capire la scala dei valori, la presenza di outlier e la forma generale dei dati.

Applichiamo subito questa tecnica alla colonna delle elevazioni dei vulcani che abbiamo appena pulito.

In [ ]:
import matplotlib.pyplot as plt

# Assicuriamoci che il DataFrame 'df' esista e non sia vuoto
if not df.empty and 'elevation_meters' in df.columns:

    # --- 1. Selezionare la colonna di interesse ---
    # Il risultato è una Pandas Series, che si comporta in modo molto simile a un array NumPy
    # quando passata a funzioni come plt.hist().
    elevations = df['elevation_meters']

    # --- 2. Creare un istogramma con Matplotlib ---
    plt.figure(figsize=(10, 6))

    plt.hist(elevations, bins=50, edgecolor='k')

    # --- 3. Personalizzare il grafico ---
    plt.title("Distribuzione delle Elevazioni dei Vulcani nel Dataset")
    plt.xlabel("Elevazione (metri)")
    plt.ylabel("Frequenza (Numero di Vulcani)")
    plt.grid(axis='y', linestyle='--', alpha=0.7)

    plt.show()

else:
    print("DataFrame vuoto o colonna 'elevation_meters' non trovata.")

### **Analisi di Colonne Categoriche: `.value_counts()`**

Abbiamo visto come l'istogramma sia perfetto per analizzare la distribuzione di dati numerici. Ma come possiamo ottenere un riepilogo rapido per le colonne che contengono **dati categorici** (testo), come la colonna `type` o `country`?

Per questo, Pandas ci offre il metodo `.value_counts()`. Applicato a una `Series` (una colonna), questo metodo fa due cose:
1.  Conta le occorrenze di ogni valore unico nella colonna.
2.  Restituisce una nuova `Series` con i valori unici come indice e i loro conteggi come valori, ordinata dal più frequente al meno frequente.

È uno degli strumenti più utili per l'analisi esplorativa dei dati.

In [ ]:
# Assicuriamoci che il DataFrame non sia vuoto
if not df.empty:

    # Usiamo .value_counts() per vedere quali sono i tipi di vulcano più comuni
    print("Conteggio dei vulcani per tipo (`type`):")
    type_counts = df['type'].value_counts()

    display(type_counts.head(10)) # Mostriamo i 10 più comuni

else:
    print("Il DataFrame è vuoto.")

### **Selezione di Righe: `.loc` e `.iloc`**

Oltre a selezionare colonne (selezione "verticale"), una delle operazioni più comuni è selezionare righe specifiche (selezione "orizzontale"). Pandas offre due metodi principali per questo, ed è fondamentale capirne la differenza:

1.  **`.iloc` (Integer Location)**
    *   Questo metodo seleziona le righe in base alla loro **posizione numerica intera**, partendo da 0.
    *   Si comporta esattamente come l'indicizzazione delle liste Python o degli array NumPy.
    *   È utile quando si vuole "la prima riga", "la decima riga", o un "intervallo di righe dalla quinta alla decima".
    *   La sintassi per lo slicing (`start:stop`) con `.iloc` **esclude** l'elemento `stop`, proprio come in Python standard.

2.  **`.loc` (Location)**
    *   Questo metodo seleziona le righe in base alla loro **etichetta dell'indice** (`index`).
    *   Di default, l'indice di un DataFrame è una sequenza di numeri interi (0, 1, 2, ...), quindi in un DataFrame appena caricato, `.loc[0]` e `.iloc[0]` spesso fanno la stessa cosa.
    *   La sintassi per lo slicing (`start_label:stop_label`) con `.loc`, a differenza di `.iloc`, **include** l'elemento `stop_label`.

**Rendere l'Indice Significativo: `.set_index()`**
Il vero potere di `.loc` emerge quando l'indice non è solo una sequenza di numeri, ma contiene informazioni significative, come l'ID di un campione o, nel nostro caso, il nome di un vulcano.

Per trasformare una delle nostre colonne in un indice, usiamo il metodo `.set_index('nome_colonna')`. Questa operazione:
1.  Prende la colonna specificata.
2.  La sposta dal corpo del DataFrame per usarla come etichette per le righe (l'indice).
3.  Restituisce un **nuovo** DataFrame con questo nuovo indice.

Una volta impostato un indice di questo tipo, possiamo usare `.loc` per selezionare le righe in modo molto più leggibile e intuitivo, usando i nomi stessi dei vulcani.

**In sintesi:**
*   **`.iloc` -> Posizione numerica (es. `df.iloc[5]`).**
*   **`.loc`  -> Etichetta dell'indice (es. `df.loc['Vesuvius']`).**

Vediamo entrambi in azione.

In [ ]:
# Assicuriamoci che il DataFrame non sia vuoto
if not df.empty:
    print("--- Esempi con .iloc (selezione per posizione) ---\n")

    # 1. Selezionare la prima riga (posizione 0)
    # Il risultato è una Series, dove l'indice sono i nomi delle colonne.
    prima_riga = df.iloc[0]
    print("Prima riga del DataFrame (come Series):")
    display(prima_riga)

    # 2. Selezionare l'ultima riga (posizione -1)
    ultima_riga = df.iloc[-1]
    print("\nUltima riga del DataFrame:")
    display(ultima_riga)

    # 3. Selezionare un intervallo di righe (dalla posizione 5 alla 10 esclusa)
    # Il risultato è un nuovo DataFrame
    righe_5_a_9 = df.iloc[5:10]
    print("\nRighe dalla posizione 5 alla 9:")
    display(righe_5_a_9)

else:
    print("Il DataFrame è vuoto.")

In [ ]:
# Per dimostrare la vera potenza di .loc, dobbiamo prima impostare un indice significativo.
# Usiamo la colonna 'name' (il nome del vulcano) come nuovo indice del DataFrame.
# Il metodo .set_index() crea un nuovo DataFrame con l'indice specificato.
if not df.empty:
    print("Imposto la colonna 'name' come nuovo indice del DataFrame...")
    df_indexed = df.set_index('name')
    display(df_indexed.head())

    print("\n--- Esempi con .loc (selezione per etichetta) ---\n")

    try:
        # 1. Selezionare una singola riga usando l'etichetta dell'indice
        vesuvius_data = df_indexed.loc['Vesuvius']
        print("Dati per il vulcano 'Vesuvius':")
        display(vesuvius_data)

        # 2. Selezionare più righe passando una lista di etichette
        etna_stromboli = df_indexed.loc[['Etna', 'Stromboli']]
        print("\nDati per 'Etna' e 'Stromboli':")
        display(etna_stromboli)

    except KeyError as e:
        print(f"ERRORE: Una delle etichette non è stata trovata nell'indice. Dettagli: {e}")
    except Exception as e:
        print(f"Si è verificato un errore: {e}")

else:
    print("Il DataFrame è vuoto.")

### **Selezione Condizionale (Filtri Booleani)**

Spesso non vogliamo selezionare righe in base alla loro posizione o a un'etichetta fissa, ma in base a una **condizione sui loro dati**. Ad esempio, "tutti i vulcani più alti di 3000 metri" o "tutti i vulcani in Italia".

Questa operazione, nota come **selezione condizionale** o **filtraggio booleano**, è una delle tecniche più potenti di Pandas. La logica è identica al *masking booleano* che abbiamo già visto in NumPy.

Il processo si articola in due passaggi:

1.  **Creare la Condizione (la Maschera):**
    Si scrive una condizione logica che viene applicata a una colonna (una `Series`). Pandas esegue il confronto elemento per elemento (in modo vettorizzato) e restituisce una nuova `Series` contenente solo valori booleani (`True` o `False`).
    *   `True` dove la condizione è soddisfatta.
    *   `False` dove la condizione non è soddisfatta.

2.  **Applicare il Filtro:**
    Si usa questa `Series` booleana all'interno delle parentesi quadre del DataFrame. Pandas restituirà un nuovo DataFrame contenente solo le **righe** in cui la maschera era `True`.

**Combinare Più Condizioni**
Per creare filtri più complessi, possiamo combinare più condizioni usando gli operatori logici:
*   `&` per **AND** (entrambe le condizioni devono essere vere).
*   `|` per **OR** (almeno una delle condizioni deve essere vera).
*   `~` per **NOT** (inverte la condizione).

**ATTENZIONE:** Quando si combinano più condizioni, ogni singola condizione **deve essere racchiusa tra parentesi `()`**. Questo è un requisito di sintassi di Pandas.
`df[(condizione_1) & (condizione_2)]`

In [ ]:
# Assicuriamoci che il DataFrame non sia vuoto
if not df.empty:

    # --- Esempio 1: Filtro Semplice ---
    # Vogliamo selezionare tutti i vulcani che si trovano in Giappone.

    # 1. Creiamo la condizione (la maschera booleana)
    condizione_giappone = df['country'] == 'Japan'

    print("Maschera booleana per i vulcani in Giappone (prime 10 righe):")
    display(condizione_giappone.head(10))

    # 2. Applichiamo il filtro
    vulcani_giapponesi = df[condizione_giappone]

    print("\nDataFrame filtrato: solo i vulcani in Giappone (prime 5 righe):")
    display(vulcani_giapponesi.head())

else:
    print("Il DataFrame è vuoto.")

In [ ]:
# Assicuriamoci che il DataFrame non sia vuoto
if not df.empty:

    # --- Esempio 2: Filtro Complesso (due condizioni) ---
    # Vogliamo selezionare tutti gli Stratovulcani ('Stratovolcano')
    # che hanno un'elevazione superiore a 3000 metri.

    # 1. Definiamo le due condizioni separatamente
    condizione_tipo = df['type'] == 'Stratovolcano'
    condizione_elevazione = df['elevation_meters'] > 3000

    # 2. Combiniamo le condizioni con l'operatore AND (&)
    # NOTA LE PARENTESI OBBLIGATORIE attorno a ogni condizione!
    stratovulcani_alti = df[ (condizione_tipo) & (condizione_elevazione) ]

    print("Vulcani che sono 'Stratovolcano' E hanno elevazione > 3000 metri:")

    # Mostriamo solo alcune colonne per una visualizzazione più pulita
    display(stratovulcani_alti[['name', 'country', 'type', 'elevation_meters']])

else:
    print("Il DataFrame è vuoto.")

## **Esercizio Finale: Analisi dei Vulcani Potenzialmente Pericolosi**

È il momento di mettere insieme tutto ciò che abbiamo imparato su Pandas (e di richiamare qualcosa da Matplotlib) per condurre un'analisi completa.

**Scenario:**
Vogliamo analizzare il catalogo dei vulcani per identificare gli **Stratovulcani** che hanno avuto un'**eruzione datata nell'Olocene**. Questi sono candidati interessanti per studi di pericolosità vulcanica.

**Compiti da Svolgere:**

1.  **Preparazione del DataFrame:**
    *   Assicurati che il DataFrame `df` sia caricato e che i nomi delle colonne siano stati puliti (tutti minuscoli e con `_` al posto degli spazi). Se necessario, riesegui le celle precedenti.

2.  **Filtraggio dei Vulcani:**
    *   Crea un nuovo DataFrame chiamato `dangerous_strato` che contenga solo i vulcani che soddisfano **entrambe** le seguenti condizioni:
        *   La colonna `type` deve essere `'Stratovolcano'`.
        *   La colonna `activity_evidence` deve essere `'Eruption Dated'`.
    *   Stampa il numero di vulcani trovati. **Suggerimento:** puoi usare `len(dangerous_strato)` o l'attributo `.shape`.

3.  **Analisi Statistica:**
    *   Usa il metodo `.describe()` sul DataFrame `dangerous_strato` per ottenere le statistiche principali (in particolare, guarda l'elevazione). Qual è l'elevazione massima tra questi vulcani?

4.  **Analisi Geografica:**
    *   Usa il metodo `.value_counts()` sulla colonna `country` del DataFrame `dangerous_strato` per scoprire quali sono i 5 paesi con il maggior numero di questi vulcani. **Suggerimento:** puoi concatenare `.head(5)` dopo `.value_counts()`.

5.  **Visualizzazione (Integrazione con Matplotlib):**
    *   Crea un **istogramma** delle elevazioni (`elevation_meters`) dei vulcani nel DataFrame `dangerous_strato`.
    *   Personalizza il grafico con un titolo ("Distribuzione delle Elevazioni degli Stratovulcani Attivi"), etichette per gli assi e una griglia.

In [ ]:
# --- ESERCIZIO FINALE ---

# 1. Preparazione del DataFrame
# Assicuriamoci che il DataFrame 'df' esista e sia pulito.
if 'df' in locals() and not df.empty:
    print("DataFrame 'df' pronto per l'analisi.")
else:
    print("ERRORE: DataFrame 'df' non trovato o vuoto. Esegui le celle precedenti.")

# 2. Filtraggio dei Vulcani
# Scrivi qui il tuo codice per creare 'dangerous_strato' e stamparne la dimensione.


# 3. Analisi Statistica
# Scrivi qui il tuo codice per usare .describe() su 'dangerous_strato'.


# 4. Analisi Geografica
# Scrivi qui il tuo codice per trovare i top 5 paesi con .value_counts().


# 5. Visualizzazione (Istogramma)
# Assicurati di aver importato matplotlib.pyplot as plt
import matplotlib.pyplot as plt
# Scrivi qui il tuo codice per creare l'istogramma delle elevazioni.

In [ ]:
# --- ESERCIZIO FINALE: SOLUZIONE ---

# 1. Preparazione del DataFrame
# Ci assicuriamo che il DataFrame 'df' esista e non sia vuoto.
if 'df' in locals() and not df.empty:
    print("DataFrame 'df' pronto per l'analisi.\n")

    # 2. Filtraggio dei Vulcani
    print("--- 2. Filtraggio dei vulcani ---")
    # Definiamo le due condizioni booleane
    condizione_tipo = df['type'] == 'Stratovolcano'
    condizione_attivita = df['activity_evidence'] == 'Eruption Dated'

    # Applichiamo entrambe le condizioni con l'operatore AND (&)
    dangerous_strato = df[condizione_tipo & condizione_attivita]

    print(f"Trovati {len(dangerous_strato)} stratovulcani con eruzioni datate.\n")
    # In alternativa, usando .shape: print(f"Trovati {dangerous_strato.shape[0]} ...")


    # 3. Analisi Statistica
    print("--- 3. Analisi statistica delle elevazioni ---")
    # Usiamo .describe() per ottenere un riepilogo statistico
    # Selezioniamo solo la colonna 'elevation_meters' per un output più pulito
    elevation_stats = dangerous_strato['elevation_meters'].describe()
    display(elevation_stats)
    print(f"L'elevazione massima tra questi vulcani è: {elevation_stats['max']} metri.\n")


    # 4. Analisi Geografica
    print("--- 4. Analisi geografica (Top 5 Paesi) ---")
    # Usiamo .value_counts() sulla colonna 'country' e concateniamo .head(5)
    top_5_countries = dangerous_strato['country'].value_counts().head(5)

    print("I 5 paesi con il maggior numero di stratovulcani attivi in questo dataset sono:")
    display(top_5_countries)
    print("\n")


    # 5. Visualizzazione (Istogramma)
    print("--- 5. Visualizzazione della distribuzione delle elevazioni ---")
    # Assicuriamoci che matplotlib sia importato
    import matplotlib.pyplot as plt

    # Selezioniamo la colonna delle elevazioni dal nostro DataFrame filtrato
    elevations_to_plot = dangerous_strato['elevation_meters']

    # Creiamo la figura e l'istogramma
    plt.figure(figsize=(10, 6))
    plt.hist(elevations_to_plot, bins=30, edgecolor='black', alpha=0.7)

    # Aggiungiamo le personalizzazioni
    plt.title("Distribuzione delle Elevazioni degli Stratovulcani Attivi")
    plt.xlabel("Elevazione (metri)")
    plt.ylabel("Frequenza (Numero di Vulcani)")
    plt.grid(axis='y', linestyle='--', alpha=0.7)

    # Aggiungiamo una linea per la media
    media_elevazione = elevation_stats['mean']
    plt.axvline(media_elevazione, color='red', linestyle='--', linewidth=2, label=f'Media: {media_elevazione:.0f} m')
    plt.legend()

    # Mostriamo il grafico
    plt.show()

else:
    print("ERRORE: DataFrame 'df' non trovato o vuoto. Esegui le celle precedenti per caricare e pulire i dati.")

---
## **(Opzionale) Esercizio per Casa: Analisi Geochimica di Rocce Ignee**

Per fare ulteriore pratica con il flusso di lavoro che abbiamo visto oggi (Download -> Caricamento -> Filtro -> Visualizzazione), ecco un esercizio da svolgere in autonomia.

L'obiettivo è scaricare un nuovo dataset geochimico da Kaggle e analizzare la distribuzione della Silice (SiO₂) per un tipo specifico di roccia.

**Dataset da Utilizzare:**
*   **Nome su Kaggle:** Geochemical Variations in Igneous Rocks - Mining
*   **Identificativo per `kagglehub`:** `cristianminas/geochemical-variations-in-igneous-rocks-mining`

**Compiti da Svolgere:**

1.  **Download e Caricamento:**
    *   In una nuova cella, usa `kagglehub.dataset_download()` con l'identificativo fornito per scaricare il dataset.
    *   Usa `os.listdir()` per scoprire il nome del file `.csv` contenuto nel dataset.
    *   Carica il file in un nuovo DataFrame Pandas (es. `df_igneous`).

2.  **Ispezione e Pulizia:**
    *   Usa `.head()` e `.info()` per ispezionare il nuovo DataFrame.
    *   Pulisci i nomi delle colonne, rendendoli minuscoli e usando `_` al posto degli spazi o altri caratteri (come abbiamo fatto in lezione).

3.  **Filtraggio:**
    *   Crea un nuovo DataFrame chiamato `andesites` che contenga solo le righe dove il tipo di roccia (`rock name`) è `'Andesite'`. **Attenzione:** controlla il nome esatto della colonna e il valore testuale dopo l'ispezione! Potrebbe essere necessario usare `.str.contains('Andesite', case=False)` se la formattazione non è consistente.

4.  **Selezione e Analisi:**
    *   Dal DataFrame `andesites`, seleziona la colonna relativa alla SiO₂.
    *   Quanti campioni di andesite ci sono nel dataset?
    *   Qual è il contenuto medio di SiO₂ per queste andesiti? (Usa il metodo `.mean()`).

5.  **Visualizzazione:**
    *   Crea un **istogramma** della colonna per i soli campioni di andesite.
    *   Personalizza il grafico con un titolo, etichette per gli assi (`SiO₂ (%)`, `Frequenza`) e una griglia.
    *   (Bonus) Aggiungi una linea verticale (`axvline`) per indicare la media di SiO₂ che hai calcolato.